In [ ]:
import numpy as np
import pickle

## Complete Trajectory

#### NCP_1KX5

In [ ]:
conditional_activity_matrix_6µs = np.load('Conditional_Activity_matrix_1.npy')
conditional_activity_matrix_6µs[np.isnan(conditional_activity_matrix_6µs)] = 0

## Split Trajectory in half i.e last 3 µs

In [ ]:
conditional_activity_matrix_3µs = np.load('Conditional_Activity_matrix_3µs.npy')
conditional_activity_matrix_3µs[np.isnan(conditional_activity_matrix_3µs)] = 0
conditional_activity_matrix_3µs.shape

(1076, 1076)

In [ ]:
def ca_similarity(A, B, tau=0.0, ignore_diag=True):
    A = np.asarray(A); B = np.asarray(B)
    assert A.shape == B.shape

    N = A.shape[0]
    off = ~np.eye(N, dtype=bool)

    mask = np.isfinite(A) & np.isfinite(B)
    if ignore_diag:
        mask = mask & off

    # Pattern
    r = np.corrcoef(A[mask].ravel(), B[mask].ravel())[0, 1] if mask.any() else np.nan

    # Magnitude
    fro = np.linalg.norm(A - B, ord="fro")
    mse = np.mean((A - B)**2)

    # Cosine
    a = A[mask].ravel()
    b = B[mask].ravel()
    cos = (np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))) if a.size else np.nan

    # Topology overlap
    EA = (A > tau); EB = (B > tau)
    if ignore_diag:
        EA[~off] = False
        EB[~off] = False
    inter = np.logical_and(EA, EB).sum()
    uni = np.logical_or(EA, EB).sum()
    jacc = inter / uni if uni else np.nan

    # Node strengths
    out_r = np.corrcoef(A.sum(axis=1), B.sum(axis=1))[0, 1]
    in_r  = np.corrcoef(A.sum(axis=0), B.sum(axis=0))[0, 1]

    return {
        "pearson_pattern": r,
        "cosine_pattern": cos,
        "frobenius_diff": fro,
        "mse_diff": mse,
        "jaccard_edges": jacc,
        "out_strength_corr": out_r,
        "in_strength_corr": in_r,
    }

# Example:
metrics = ca_similarity(conditional_activity_matrix_6µs, conditional_activity_matrix_3µs, tau=0.0, ignore_diag=False)
print(metrics)


{'pearson_pattern': 0.9136696691562609, 'cosine_pattern': 0.9136775234061371, 'frobenius_diff': 703.0876704558082, 'mse_diff': 0.42696710965417783, 'jaccard_edges': 0.8516735694362269, 'out_strength_corr': 0.925882126590486, 'in_strength_corr': 0.9365648567721925}
